In [3]:
# dependencies

import os
import shutil
import math
import random
import csv
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from skimage import io
import torchvision.datasets as datasets
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.io import read_image
import skimage.io as io

In [4]:
# connecting to drive to save models
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Data

### (600,450)
creates dataloader, that holds a tensor representation of the whole (600,450) image. This is to big to work with.v

In [5]:
# Create train/test split (80%)/(20%):

source_folder = 'Data/HAM100000_images/'
test_folder = 'Data/HAM100000_images_test/'
train_folder = 'Data/HAM100000_images_train/'

file_list = os.listdir(source_folder)

number_test_images = math.ceil(0.2 * len(file_list))
number_train_images = len(file_list) - number_test_images
# TODO: Set manual seed and select number_test_images in and rest in 'Data/HAM100000_images_train' 
seed_value = 42
random.seed(seed_value)

random_numbers = set()

while len(random_numbers) <= number_test_images:
    random_numbers.add(random.randint(0, len(file_list)-1))

for index, file in enumerate(file_list):
    if index in random_numbers:
        source_path = os.path.join(source_folder, file)
        destination_path = os.path.join(train_folder, file)
        shutil.copy(source_path, destination_path)
    else:
        source_path = os.path.join(source_folder, file)
        destination_path = os.path.join(test_folder, file)
        shutil.copy(source_path, destination_path)

FileNotFoundError: ignored

In [ ]:
# Create annotations file for given train/test split

test_directory = 'Data/HAM100000_images_test/'

with open('Data/HAM10000_labels.csv') as file:
    csv_reader = csv.reader(file)

    print(next(csv_reader)[0])
    write_test = []
    write_train = []

    for row in csv_reader:
        path_image = os.path.join(test_directory, row[0])
        path_image = path_image + '.jpg'
        if os.path.isfile(path_image):
            write_test.append([row[0], row[1]])
        else:
            write_train.append([row[0], row[1]])
    
with open('Data/HAM100000_train_label.csv', 'w', newline='') as file:
    csv_writer = csv.writer(file)
    for row in write_train:
        csv_writer.writerow(row)


with open('Data/HAM100000_test_label.csv', 'w', newline='') as file:
    csv_writer = csv.writer(file)
    for row in write_test:
        csv_writer.writerow(row)

In [ ]:
# show an image
root = 'Data\HAM100000_images'
frame = pd.read_csv('Data\HAM10000_labels.csv')

image_path = os.path.join(root, frame.iloc[0,0])
image_path = image_path + '.jpg'
with Image.open(image_path) as image:
    image.show()

print(image_path)

Data\HAM100000_images\ISIC_0025030.jpg


In [ ]:
# calculate mean and std of dataset, all images have size (600, 450), after transforming images to tensors they have size (3, 600, 450)
file_list = os.listdir('Data\HAM100000_images')
mean = [0, 0, 0]
std = [0, 0, 0]

for file_name in file_list:
    image_path = os.path.join('Data\HAM100000_images', file_name)
    image = io.imread(image_path)
    mean[0] += image[:, :, 0].mean()
    mean[1] += image[:, :, 1].mean()
    mean[2] += image[:, :, 2].mean()
    std[0] += image[:, :, 0].std()
    std[1] += image[:, :, 1].std()
    std[2] += image[:, :, 2].std()    
for i in [0, 1, 2]:
    mean[i] /= len(file_list)
    std[i] /= len(file_list)

print(mean, std)

[194.69792020561766, 139.26262746509832, 145.48524135685338] [22.855094582223956, 30.168411555547745, 33.903190491317204]


In [ ]:
# load data

train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)




In [ ]:
# load dataset
batch_size = 32
shuffle = True

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[194.69792020561766, 139.26262746509832, 145.48524135685338], std=[22.855094582223956, 30.168411555547745, 33.903190491317204])
])

# to use these data needs to be in class folders

train_dataset = datasets.ImageFolder(root='Data/HAM100000_images_train/', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

test_dataset = datasets.ImageFolder(root='Data/HAM100000_images_test/', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
test = next(iter(train_loader))
print(type(test))
print(len(test))
print(type(test[0]))
print(test[0].shape)
print(type(test[1]))
print(test[1].shape)

<class 'list'>
2
<class 'torch.Tensor'>
torch.Size([32, 3, 450, 600])
<class 'torch.Tensor'>
torch.Size([32])


### (28,28)
With the dataset there comes a csv containing the same images compressed to (28,28), in a csv file. This is better to work with.

In [6]:
# generate train/test split
random.seed(42)
test_split  = set()
while len(test_split) < 2000:
    test_split.add(random.randint(0, 10015-1)) # HAM10000 has 10015 images

#read csv file into list

with open('drive/MyDrive/SkinCancer/hmnist_28_28_RGB.csv') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    data = []
    for row in csv_reader:
        data.append(row)

In [7]:
from torch.utils.data import Dataset

class CSV_Data(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [82]:
batch_size = 16
shuffle = True

train_input = []
train_labels = []
test_input = []
test_labels = []

for index, image in enumerate(data):
    image = [float(pixel) for pixel in image]
    if index in test_split:
        test_input.append(torch.tensor([[image[:-1][0:28*28][i*28:(i+1)*28] for i in range(28)], [image[:-1][28*28:2*28*28][i*28:(i+1)*28] for i in range(28)], [image[:-1][2*28*28:][i*28:(i+1)*28] for i in range(28)]], dtype=torch.float32)) # 3 channels
        test_labels.append(torch.tensor(image[-1], dtype=torch.long))
    else:
        train_input.append(torch.tensor([[image[:-1][0:28*28][i*28:(i+1)*28] for i in range(28)], [image[:-1][28*28:2*28*28][i*28:(i+1)*28] for i in range(28)], [image[:-1][2*28*28:][i*28:(i+1)*28] for i in range(28)]], dtype=torch.float32))
        train_labels.append(torch.tensor(image[-1], dtype=torch.long))
train_loader = DataLoader(CSV_Data(train_input, train_labels), batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(CSV_Data(test_input, test_labels), batch_size=batch_size, shuffle=False)
print(len(train_loader))
print(len(test_loader))

print(len(train_loader)*batch_size)
print(len(test_loader)*batch_size)

501
125
8016
2000


## random Baseline

In [98]:
baseline = net(seed=17) # depending on the initial seed, the model mighz classify all images as mel, achieving relatively high accuracy since the class is relatively large
test_model(baseline, train_loader)
print("---")
test_model(baseline, test_loader)

accuracy for akiec: 163/261 | accuracy in %: 62.45210727969349
accuracy for bcc: 6/414 | accuracy in %: 1.4492753623188406
accuracy for bkl: 0/904 | accuracy in %: 0.0
accuracy for df: 0/86 | accuracy in %: 0.0
accuracy for mel: 0/5351 | accuracy in %: 0.0
accuracy for nv: 0/115 | accuracy in %: 0.0
accuracy for vasc: 399/884 | accuracy in %: 45.13574660633484
overall accuracy: 568 / 8015 | accuracy in percent % 7.0867124142233315
---
accuracy for akiec: 44/66 | accuracy in %: 66.66666666666666
accuracy for bcc: 1/100 | accuracy in %: 1.0
accuracy for bkl: 0/195 | accuracy in %: 0.0
accuracy for df: 0/29 | accuracy in %: 0.0
accuracy for mel: 0/1354 | accuracy in %: 0.0
accuracy for nv: 0/27 | accuracy in %: 0.0
accuracy for vasc: 103/229 | accuracy in %: 44.97816593886463
overall accuracy: 148 / 2000 | accuracy in percent % 7.4


## Model 0

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [93]:
class net(nn.Module):
    def __init__(self, dropout = False, seed=17):
        super(net, self).__init__()
        torch.manual_seed(seed)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=0)
        
        self.fc1 = nn.Linear(32 * 6 * 6, 512)
        self.fc2 = nn.Linear(512, 100)
        self.fc3 = nn.Linear(100, 7)

        # less droupout in deeper layers
        self.dropout = False 
        self.dropout_3 = nn.Dropout(p=0.3) 
        self.dropout_5 = nn.Dropout(p=0.5)
    def forward(self, x, train=True):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = x.view(-1, 32 * 6 * 6)
        x = self.fc1(x)
        x = F.relu(x)
        if train and self.dropout:
            x = self.dropout_5(x)    
        x = self.fc2(x)
        x = F.relu(x)
        if train and self.dropout:
            x = self.dropout_3(x)
        x = self.fc3(x)
        
        return x

classes = ('akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc')

In [54]:
# train loop
def train_model(model, data_loader, epochs, lr=0.1, optim=None):
  if optim is None:
    optimizer = torch.optim.SGD(params=model.parameters(), lr=lr)
  else:
    optimizer = optim
  loss_fn = nn.CrossEntropyLoss()
  
  model.to(device)
  for epoch in range(epochs):
    epoch_loss = 0
    for batch, (X, y) in enumerate(data_loader):
      X = X.to(device)
      y = y.to(device)
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      epoch_loss += loss.item()
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print(f"average batch loss in {epoch+1}: {epoch_loss/len(data_loader)}")

In [63]:
# test loop
def test_model(model, data_loader):
  model.to(device)
  with torch.no_grad():
    right, wrong = np.array([0 for i in range(10)]), np.array([0 for i in range(10)])
    for batch, (X,y) in enumerate(data_loader):
      X = X.to(device)
      y = y.to(device)
      y_pred = model(X, train=False)
      y_pred = torch.argmax(y_pred, dim=1)
      for i in range(len(y_pred)):
        if y_pred[i]==y[i]:
          right[y[i]] += 1
        else:
         wrong[y[i]] += 1

  for i in range(len(classes)):
    print(f"accuracy for {classes[i]}: {right[i]}/{right[i]+wrong[i]} | accuracy in %: {right[i]/(right[i]+wrong[i])*100}")
  
  print(f"overall accuracy: {np.sum(right)} / {np.sum(right) + np.sum(wrong)} | accuracy in percent % {100*np.sum(right)/(np.sum(right) + np.sum(wrong))}")

In [83]:
model_0 = net()
for i in range(10):
  print(f"iteration:{i+1}")
  train_model(model_0, train_loader, epochs=10, lr=0.01/(i+1))

iteration:1
average batch loss in 1: 1.049974169976221
average batch loss in 2: 0.9300510172180073
average batch loss in 3: 0.884845799433733
average batch loss in 4: 0.8498187974511744
average batch loss in 5: 0.8199817973101686
average batch loss in 6: 0.7759172544032038
average batch loss in 7: 0.7514441572204084
average batch loss in 8: 0.7109242270211735
average batch loss in 9: 0.6758476552551616
average batch loss in 10: 0.6297376193626674
iteration:2
average batch loss in 1: 0.5019356618800801
average batch loss in 2: 0.4454693213224173
average batch loss in 3: 0.4076186245430016
average batch loss in 4: 0.35960420145246086
average batch loss in 5: 0.31475842865314074
average batch loss in 6: 0.2827510247955065
average batch loss in 7: 0.2474949287451402
average batch loss in 8: 0.21039273906053005
average batch loss in 9: 0.1909466605845415
average batch loss in 10: 0.16382146018283988
iteration:3
average batch loss in 1: 0.07398186001077131
average batch loss in 2: 0.04608437

In [84]:
test_model(model_0, train_loader)
print("---")
test_model(model_0, test_loader)

accuracy for akiec: 261/261 | accuracy in %: 100.0
accuracy for bcc: 414/414 | accuracy in %: 100.0
accuracy for bkl: 904/904 | accuracy in %: 100.0
accuracy for df: 86/86 | accuracy in %: 100.0
accuracy for mel: 5351/5351 | accuracy in %: 100.0
accuracy for nv: 115/115 | accuracy in %: 100.0
accuracy for vasc: 884/884 | accuracy in %: 100.0
overall accuracy: 8015 / 8015 | accuracy in percent % 100.0
---
accuracy for akiec: 12/66 | accuracy in %: 18.181818181818183
accuracy for bcc: 44/100 | accuracy in %: 44.0
accuracy for bkl: 84/195 | accuracy in %: 43.07692307692308
accuracy for df: 6/29 | accuracy in %: 20.689655172413794
accuracy for mel: 1210/1354 | accuracy in %: 89.36484490398819
accuracy for nv: 13/27 | accuracy in %: 48.148148148148145
accuracy for vasc: 72/229 | accuracy in %: 31.4410480349345
overall accuracy: 1441 / 2000 | accuracy in percent % 72.05


## Model 1

Problems with model 0:


1.   Overfitting
2.   Number of samples is not evenly distributed, which leads to the model being biased in favour of more frequent classes.

Mittigation:
1. data augmentation
2. regularization
3. batch normalization






In [80]:
# data augmentation
# first we just rotate all images and add them to the train/test set, later we might only pictures such that the classes are all about the same size

batch_size = 16
shuffle = True

aug_train_inputs = []
aug_train_labels = []
aug_test_inputs = []
aug_test_labels = []

for index, image in enumerate(data):
    image = [float(pixel) for pixel in image]
    tensor_image = torch.tensor([[image[:-1][0:28*28][i*28:(i+1)*28] for i in range(28)], [image[:-1][28*28:2*28*28][i*28:(i+1)*28] for i in range(28)], [image[:-1][2*28*28:][i*28:(i+1)*28] for i in range(28)]], dtype=torch.float32)
    if index in test_split:
      aug_test_inputs.append(tensor_image)
      aug_test_labels.append(torch.tensor(image[-1], dtype=torch.long))
      for i in [1,2,3]:
        aug_test_inputs.append(torch.rot90(tensor_image, i, [1, 2]))
        aug_test_labels.append(torch.tensor(image[-1], dtype=torch.long))
    else:
      aug_train_inputs.append(tensor_image)
      aug_train_labels.append(torch.tensor(image[-1], dtype=torch.long))
      for i in [1,2,3]:
        aug_train_inputs.append(torch.rot90(tensor_image, i, [1, 2]))
        aug_train_labels.append(torch.tensor(image[-1], dtype=torch.long))
aug_train_loader = DataLoader(CSV_Data(aug_train_inputs, aug_train_labels), batch_size=batch_size, shuffle=shuffle)
aug_test_loader = DataLoader(CSV_Data(aug_test_inputs, aug_test_labels), batch_size=batch_size, shuffle=False)
print(len(aug_train_loader))
print(len(aug_test_loader))
print(len(aug_train_loader)*batch_size)
print(len(aug_test_loader)*batch_size)

2004
500
32064
8000


In [103]:
aug_model_0 = net()
for i in range(0,10):
  print(f"iteration:{i+1}")
  train_model(aug_model_0, aug_train_loader, epochs=10, lr=0.01/(i+1))

iteration:1
average batch loss in 1: 1.0428919268998795
average batch loss in 2: 0.9222879297139045
average batch loss in 3: 0.8845254626281247
average batch loss in 4: 0.849607192314492
average batch loss in 5: 0.8226646314823104
average batch loss in 6: 0.7968466394839292
average batch loss in 7: 0.7703361720471325
average batch loss in 8: 0.7445546372936752
average batch loss in 9: 0.7244395885891662
average batch loss in 10: 0.699477148084286
iteration:2
average batch loss in 1: 0.5850905268239285
average batch loss in 2: 0.5421568215592297
average batch loss in 3: 0.5040901447190883
average batch loss in 4: 0.4765532100777664
average batch loss in 5: 0.4473450482021757
average batch loss in 6: 0.42238394958812914
average batch loss in 7: 0.40113424159541816
average batch loss in 8: 0.37279567419237153
average batch loss in 9: 0.34962549436504686
average batch loss in 10: 0.34024955724791706
iteration:3
average batch loss in 1: 0.22414609870525773
average batch loss in 2: 0.1835354

In [104]:
test_model(aug_model_0, aug_train_loader)
print("---")
test_model(aug_model_0, aug_test_loader)

accuracy for akiec: 1044/1044 | accuracy in %: 100.0
accuracy for bcc: 1656/1656 | accuracy in %: 100.0
accuracy for bkl: 3616/3616 | accuracy in %: 100.0
accuracy for df: 344/344 | accuracy in %: 100.0
accuracy for mel: 21404/21404 | accuracy in %: 100.0
accuracy for nv: 460/460 | accuracy in %: 100.0
accuracy for vasc: 3536/3536 | accuracy in %: 100.0
overall accuracy: 32060 / 32060 | accuracy in percent % 100.0
---
accuracy for akiec: 57/264 | accuracy in %: 21.59090909090909
accuracy for bcc: 146/400 | accuracy in %: 36.5
accuracy for bkl: 253/780 | accuracy in %: 32.43589743589744
accuracy for df: 24/116 | accuracy in %: 20.689655172413794
accuracy for mel: 4777/5416 | accuracy in %: 88.2016248153619
accuracy for nv: 49/108 | accuracy in %: 45.370370370370374
accuracy for vasc: 266/916 | accuracy in %: 29.03930131004367
overall accuracy: 5572 / 8000 | accuracy in percent % 69.65


In [105]:
aug_dropout_model_0 = net(dropout=True)
for i in range(0,10,2):
  print(f"iteration:{i+1}")
  train_model(aug_dropout_model_0, aug_train_loader, epochs=10, lr=0.01/(i+1))

iteration:1
average batch loss in 1: 1.0346965776187573
average batch loss in 2: 0.904344401606721
average batch loss in 3: 0.8592819171453903
average batch loss in 4: 0.8205597791263086
average batch loss in 5: 0.7864577781981574
average batch loss in 6: 0.7534084853729207
average batch loss in 7: 0.7192131275322265
average batch loss in 8: 0.6834684544635152
average batch loss in 9: 0.6527785609426078
average batch loss in 10: 0.6146692164993274
iteration:3
average batch loss in 1: 0.44561731751391037
average batch loss in 2: 0.38072895543452745
average batch loss in 3: 0.3339132139805013
average batch loss in 4: 0.2936805110297421
average batch loss in 5: 0.25460054973575347
average batch loss in 6: 0.2271442684398856
average batch loss in 7: 0.19853924664049716
average batch loss in 8: 0.18346364349240676
average batch loss in 9: 0.15456232328598524
average batch loss in 10: 0.14655943787974485
iteration:5
average batch loss in 1: 0.0756050420117782
average batch loss in 2: 0.04382

In [106]:
test_model(aug_dropout_model_0, aug_train_loader)
print("---")
test_model(aug_dropout_model_0, aug_test_loader)

accuracy for akiec: 1044/1044 | accuracy in %: 100.0
accuracy for bcc: 1656/1656 | accuracy in %: 100.0
accuracy for bkl: 3616/3616 | accuracy in %: 100.0
accuracy for df: 344/344 | accuracy in %: 100.0
accuracy for mel: 21404/21404 | accuracy in %: 100.0
accuracy for nv: 460/460 | accuracy in %: 100.0
accuracy for vasc: 3536/3536 | accuracy in %: 100.0
overall accuracy: 32060 / 32060 | accuracy in percent % 100.0
---
accuracy for akiec: 62/264 | accuracy in %: 23.484848484848484
accuracy for bcc: 144/400 | accuracy in %: 36.0
accuracy for bkl: 292/780 | accuracy in %: 37.43589743589744
accuracy for df: 17/116 | accuracy in %: 14.655172413793101
accuracy for mel: 4806/5416 | accuracy in %: 88.7370753323486
accuracy for nv: 52/108 | accuracy in %: 48.148148148148145
accuracy for vasc: 266/916 | accuracy in %: 29.03930131004367
overall accuracy: 5639 / 8000 | accuracy in percent % 70.4875
